In [1]:
import os
import requests
import boto3
import botocore
import numpy as np
import dask
from dask.distributed import Client, Semaphore
import pandas as pd
from pathlib import Path

In [2]:
dask_client = Client(n_workers=64, threads_per_worker=1, memory_limit='20GB')
dask_client

/water2/pdas47/2023_01_24-river-regulation/.env/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 34615 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:34615/status,
Dashboard: http://127.0.0.1:34615/status,Workers: 64
Total threads: 64,Total memory: 1.16 TiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38693,Workers: 64
Dashboard: http://127.0.0.1:34615/status,Total threads: 64
Started: Just now,Total memory: 1.16 TiB
Comm: tcp://127.0.0.1:35561,Total threads: 1
Dashboard: http://127.0.0.1:40345/status,Memory: 18.63 GiB
Nanny: tcp://127.0.0.1:34231,


Set up S3 Access using Boto

In [3]:
era5_bucket = 'era5-pds'
client = boto3.client('s3', config=botocore.client.Config(signature_version=botocore.UNSIGNED))

In [4]:
paginator = client.get_paginator('list_objects')
result = paginator.paginate(Bucket=era5_bucket, Delimiter='/')
for prefix in result.search('CommonPrefixes'):
    print(prefix.get('Prefix'))


1979/
1980/
1981/
1982/
1983/
1984/
1985/
1986/
1987/
1988/
1989/
1990/
1991/
1992/
1993/
1994/
1995/
1996/
1997/
1998/
1999/
2000/
2001/
2002/
2003/
2004/
2005/
2006/
2007/
2008/
2009/
2010/
2011/
2012/
2013/
2014/
2015/
2016/
2017/
2018/
2019/
2020/
2021/
2022/
2023/
QA/
zarr/


In [7]:
@dask.delayed(nout=3)
def download_era5_file(bucket, aws_obj, dst_fp, sem):
    """Download ERA5 file from S3 bucket for a given date

    Args:
        date (DateTime): Date to download data for
        bucket (str): S3 bucket name to download from
        dst_dir (Path | str): Directory to save file to
        sem (Semaphore): Semaphore to limit concurrent downloads

    Returns:
        int: Download status (0 = success, 1 = failure)
    """
    with sem:
        client = boto3.client('s3', config=botocore.client.Config(signature_version=botocore.UNSIGNED))
        try:
            with open(dst_fp, 'wb') as f:
                client.download_fileobj(bucket, aws_obj, f)
            download_status = 0
        except botocore.exceptions.ClientError as e:
            download_status = 1
    
    return download_status

def retrieve_era5_precip(start_date, end_date, bucket, dst_dir, max_concurrent=4):
    """
    Retrieve ERA5 data from S3 bucket for a given date range
    """
    dst_dir = Path(dst_dir)
    date_range = pd.date_range(start_date, end_date, freq='MS')
    statuses = []
    aws_objs = []
    filenames = []
    
    # Limit concurrent downloads using a semaphore
    sem = Semaphore(max_concurrent, name='aws-era5')
    # dask.config.set('distributed.scheduler.locks.lease-timeout', '1800s')

    for date in date_range:
        prefix = f"{date.strftime('%Y/%m/')}data/"
        dst_fp = dst_dir / (date.strftime('%Y%m%d') + '.nc')
        dst_fp.parent.mkdir(parents=True, exist_ok=True)
        aws_obj = prefix + 'precipitation_amount_1hour_Accumulation.nc'
        
        status = download_era5_file(bucket, aws_obj, dst_fp, sem)

        statuses.append(status)
        aws_objs.append(aws_obj)
        filenames.append(dst_fp)

    statuses = dask.compute(*statuses)
    
    return pd.DataFrame({'date': date_range, 'download_status': statuses, 'aws_obj': aws_objs, 'filename': filenames})

In [8]:
start_date = '2007-01-01'
end_date = '2010-01-01'

era5_data_retrieval = retrieve_era5_precip(start_date, end_date, era5_bucket, Path("../../data-era5/era5"), max_concurrent=12)

In [9]:
era5_data_retrieval

,date,download_status,aws_obj,filename
0,2007-01-01,0,2007/01/data/precipitation_amount_1hour_Accumu...,../../data-era5/era5/20070101.nc
1,2007-02-01,0,2007/02/data/precipitation_amount_1hour_Accumu...,../../data-era5/era5/20070201.nc
2,2007-03-01,0,2007/03/data/precipitation_amount_1hour_Accumu...,../../data-era5/era5/20070301.nc
3,2007-04-01,0,2007/04/data/precipitation_amount_1hour_Accumu...,../../data-era5/era5/20070401.nc
4,2007-05-01,0,2007/05/data/precipitation_amount_1hour_Accumu...,../../data-era5/era5/20070501.nc
5,2007-06-01,0,2007/06/data/precipitation_amount_1hour_Accumu...,../../data-era5/era5/20070601.nc
6,2007-07-01,0,2007/07/data/precipitation_amount_1hour_Accumu...,../../data-era5/era5/20070701.nc
7,2007-08-01,0,2007/08/data/precipitation_amount_1hour_Accumu...,../../data-era5/era5/20070801.nc
8,2007-09-01,0,2007/09/data/precipitation_amount_1hour_Accumu...,../../data-era5/era5/20070901.nc
9,2007-10-01,0,2007/10/data/precipitation_amount_1hour_Accumu...,../../data-era5/era5/20071001.nc
